In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

print(train_data.head())

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/submission/titanic_submission.csv
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2     

**Process to Clean the Age data**

In [2]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
train_data['Cabin'] = train_data['Cabin'].fillna('Unknown')
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
print(train_data.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


**Transform raw data into meaningful features to improve model performance.**

In [3]:
train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess', 'Dr', 'Rev', 'Major','Master', 'Sir', 'Col', 'Don','Dona', 'Jonkheer'], 'Rare')
train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
train_data['Deck'] = train_data['Cabin'].str[0].fillna('Unknown')

train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1

train_data['AgeGroup'] = pd.cut(train_data['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teen', 'Adult', 'MidAge', 'Senior'])

print(train_data['Title'])

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886    Rare
887    Miss
888    Miss
889      Mr
890      Mr
Name: Title, Length: 891, dtype: object


**Convert categorical variables (e.g., Sex, Embarked, Title) into numerical values for modeling**

In [4]:
from sklearn.preprocessing import LabelEncoder

# Encode 'Sex' and 'Embarked'
label_encoders = {}
for col in ['Sex', 'Embarked', 'Title', 'AgeGroup', 'Deck']:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    label_encoders[col] = le  # Save encoders for test data


**Feature Selection**

In [5]:
features = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'Title', 'Deck']
X = train_data[features]
y = train_data['Survived']

**Split Data into Training/Validation Sets**

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

**Training a Model**

In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_split, y_train_split)


LogisticRegression(max_iter=1000, random_state=42)

**Evaluate performance on the validation set and optimize hyperparameters**

In [9]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_pred):.2f}")

Validation Accuracy: 0.79


**Preparing Test Data**

In [10]:
age_median = train_data['Age'].median()
test_data['Age'] = test_data['Age'].fillna(age_median)

fare_median = train_data['Fare'].median()
test_data['Fare'] = test_data['Fare'].fillna(fare_median)

embarked_mode = train_data['Embarked'].mode()[0]
test_data['Embarked'] = test_data['Embarked'].fillna(embarked_mode)

test_data['Cabin'] = test_data['Cabin'].fillna('Unknown')


In [11]:
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess', 'Dr', 'Rev', 'Major','Master', 'Sir', 'Col', 'Don','Dona', 'Jonkheer'], 'Rare')
test_data['Title'] = test_data['Title'].replace('Mlle', 'Miss')
test_data['Title'] = test_data['Title'].replace('Ms', 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
test_data['Deck'] = test_data['Cabin'].str[0].fillna('Unknown')
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1

test_data['AgeGroup'] = pd.cut(test_data['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teen', 'Adult', 'MidAge', 'Senior'])

print(test_data['Title'])

0        Mr
1       Mrs
2        Mr
3        Mr
4       Mrs
       ... 
413      Mr
414    Rare
415      Mr
416      Mr
417    Rare
Name: Title, Length: 418, dtype: object


In [12]:
from sklearn.preprocessing import LabelEncoder

# Encode 'Sex' and 'Embarked'
label_encoders = {}
for col in ['Sex', 'Embarked', 'Title', 'AgeGroup','Deck']:
    le = LabelEncoder()
    test_data[col] = le.fit_transform(test_data[col].astype(str))
    label_encoders[col] = le  # Save encoders for test data

In [13]:

training_features = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'Title','Deck']
X_test = test_data[training_features]

In [14]:

print(test_data.isnull().sum())

print("Training features:", training_features)
print("Test features:", X_test.columns.tolist())

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Title          0
Deck           0
FamilySize     0
AgeGroup       0
dtype: int64
Training features: ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'Title', 'Deck']
Test features: ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'Title', 'Deck']


Testing

In [15]:
X_test = test_data[features]
test_predictions = model.predict(X_test)
print(test_predictions)

[0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 0 0 1 0 0 1 0 0 0]


**Submission**

In [16]:
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': test_predictions
})

submission.to_csv('submission.csv', index=False)